In [6]:
#needed to import PN
from pm4py.objects.petri.importer import importer as pnml_importer
#visualize PN
from pm4py.visualization.petrinet import visualizer as pn_visualizer
#playout PN
from pm4py.simulation.playout import simulator


import pandas as pd

import json

In [10]:
def import_net(filename):
    net, im , fm = pnml_importer.apply(filename)
    return net,im,fm

def gen_log_from_net(net, im, num_traces):
    log = simulator.apply(net, im, variant=simulator.Variants.BASIC_PLAYOUT,
                          parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: num_traces})
    return log

def get_alphabet(net):
    activities = list({a.label for a in net.transitions if a.label and not '_' in a.label})
    return activities

def get_integer_map_net(net):
    return {x: i+1 for i,x in enumerate(get_alphabet(net))}

def apply_integer_map(log, map):
    return [[map[a['concept:name']] for a in t] for t in log]

def get_variants(net, im, max_loops, maxlen): #get all variants from a petri net
    variants = simulator.apply(net, im, variant=simulator.Variants.EXTENSIVE, 
                               parameters={simulator.Variants.EXTENSIVE.value.Parameters.MAX_TRACE_LENGTH: 100,
                                           simulator.Variants.EXTENSIVE.value.Parameters.MAX_MARKING_OCC: max_loops+1,
                                           simulator.Variants.EXTENSIVE.value.Parameters.MAX_TRACE_LENGTH: maxlen})
    return variants

def save_log(loglist, filename): #save a list of lists 
    df = pd.DataFrame.from_records(loglist)
    df.to_csv(filename, index=False)
    
def delete_variant(log, variant): #remove a variant from a log and return new log
    return([trace for trace in log if trace != variant])

In [11]:
modelname = "Parallel" #how do we call this model in our saves

net, im, fm = import_net('5parallel.pnml')

#get number encoding we are going to use
mapping = get_integer_map_net(net)
#save number mapping
mappingfilename = modelname + '/mapping.txt'  
with open(mappingfilename, 'w') as f:
    f.write(json.dumps(mapping))

In [12]:
#get all variants
variants = apply_integer_map(get_variants(net, im, 3, 50), mapping)

varname = modelname + '/variants.csv'
save_log(variants, varname)

<ipython-input-10-9885d9b2e22e>:21: DeprecatedWarning: apply is deprecated as of 2.2.5 and will be removed in 3.0. Use the pm4py.algo.simulation.playout package
  variants = simulator.apply(net, im, variant=simulator.Variants.EXTENSIVE,
C:\Users\u0132580\Anaconda3\lib\site-packages\pm4py\simulation\playout\simulator.py:61: UserWarning: Use the pm4py.algo.simulation.playout package
  warnings.warn("Use the pm4py.algo.simulation.playout package")


In [13]:
#go through all variants, each time save the log without the variant in question

for i in range(0,len(variants)):
    print(i)
    print(variants[i])
    new_log = apply_integer_map(gen_log_from_net(net, im, 12000), mapping)
    train_log = delete_variant(new_log, variants[i])
    print(len(train_log))
    train_filename = modelname + "/Training_logs/log_" + str(i) + ".csv"
    save_log(train_log, train_filename)

0
[6, 7, 8, 13, 3, 4, 11, 2, 1, 10, 5, 12, 9]


<ipython-input-10-9885d9b2e22e>:6: DeprecatedWarning: apply is deprecated as of 2.2.5 and will be removed in 3.0. Use the pm4py.algo.simulation.playout package
  log = simulator.apply(net, im, variant=simulator.Variants.BASIC_PLAYOUT,


11888
1
[6, 7, 8, 13, 3, 4, 11, 1, 2, 10, 5, 12, 9]
11909
2
[6, 7, 8, 13, 3, 4, 1, 11, 2, 10, 5, 12, 9]
11895
3
[6, 7, 8, 13, 3, 4, 1, 2, 11, 10, 5, 12, 9]
11885
4
[6, 7, 8, 13, 3, 4, 2, 11, 1, 10, 5, 12, 9]
11899
5
[6, 7, 8, 13, 3, 4, 2, 1, 11, 10, 5, 12, 9]
11919
6
[6, 7, 8, 13, 3, 1, 4, 11, 2, 10, 5, 12, 9]
11895
7
[6, 7, 8, 13, 3, 1, 4, 2, 11, 10, 5, 12, 9]
11899
8
[6, 7, 8, 13, 3, 1, 2, 4, 11, 10, 5, 12, 9]
11906
9
[6, 7, 8, 13, 3, 1, 2, 11, 4, 10, 5, 12, 9]
11898
10
[6, 7, 8, 13, 3, 1, 11, 4, 2, 10, 5, 12, 9]
11902
11
[6, 7, 8, 13, 3, 1, 11, 2, 4, 10, 5, 12, 9]
11905
12
[6, 7, 8, 13, 3, 2, 4, 11, 1, 10, 5, 12, 9]
11917
13
[6, 7, 8, 13, 3, 2, 4, 1, 11, 10, 5, 12, 9]
11893
14
[6, 7, 8, 13, 3, 2, 1, 4, 11, 10, 5, 12, 9]
11893
15
[6, 7, 8, 13, 3, 2, 1, 11, 4, 10, 5, 12, 9]
11887
16
[6, 7, 8, 13, 3, 2, 11, 4, 1, 10, 5, 12, 9]
11906
17
[6, 7, 8, 13, 3, 2, 11, 1, 4, 10, 5, 12, 9]
11898
18
[6, 7, 8, 13, 3, 11, 4, 2, 1, 10, 5, 12, 9]
11902
19
[6, 7, 8, 13, 3, 11, 4, 1, 2, 10, 5, 12, 9]
11